# Machine Translation

1. RNN
2. Embedding-RNN
3. Bidirectional RNN 
4. Encoder-Decoder

In [1]:
# Loading Dataset
ftr=open('/content/drive/My Drive/Projects/Machine Translation/small_vocab_fr','r')
ftrain=ftr.read()
ftrain=ftrain.split('\n')
ftr.close()
etr=open('/content/drive/My Drive/Projects/Machine Translation/small_vocab_en','r')
etrain=etr.read()
etrain=etrain.split('\n')
etr.close()

In [2]:
# Text preprocessing
# We will create sequence of 100 words
import numpy as np
import tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(ftrain)
fseq=tokenizer.texts_to_sequences(ftrain)
fpad=tf.keras.preprocessing.sequence.pad_sequences(
    fseq, maxlen=21, dtype='int32', padding='post', truncating='post',
    value=0.0)


tokenizer1=tf.keras.preprocessing.text.Tokenizer()
tokenizer1.fit_on_texts(etrain)
eseq=tokenizer1.texts_to_sequences(etrain)
epad=tf.keras.preprocessing.sequence.pad_sequences(
    eseq, maxlen=21, dtype='int32', padding='post', truncating='post',
    value=0.0)


In [3]:
# Adding the pad term = <pad>

tokenizer.word_index['<pad>']=0
tokenizer1.word_index['<pad>']=0
en_itow={i:j for j,i in tokenizer1.word_index.items()}
fr_itow={i:j for j,i in tokenizer.word_index.items()}

In [11]:
trainy=fpad.reshape(137861,21,1)
trainx=epad.reshape(137861,21,1)

# Simple RNN

In [ ]:
def acc(y_true,y_pred):
  sm=0
  total=0
  y_pred=tf.math.argmax(y_pred,axis=2)
  y_true=tf.cast(y_true,dtype=tf.int64)
  y_true=tf.reshape(y_true,[len(y_true),21])
  #print('type of ytrue={}\ntype of ypred={}\n'.format(type(y_true),type(y_pred)))
  #print('\nshape of ytrue={}\nshape of ypred={}\n'.format(y_true.shape,y_pred.shape))
  #print('True Values : \n')
  #print(K.print_tensor(y_true))
  #print('Predicted Values : \n')
  #print(K.print_tensor(y_pred))
  #print('\n\ny_true={}\ny_pred={}'.format(y_true,y_pred))
  for i in range(len(y_true)):
    mask=tf.cast(y_true[i],dtype=tf.bool)
    y=tf.boolean_mask(y_true[i],mask)
    p=tf.boolean_mask(y_pred[i],mask)
    _=y==p
    #print('\n\n Comparison tensor:\n')
    #print(K.print_tensor(_))
    tmp=tf.reduce_sum(tf.cast(_, tf.int32))
    #print(K.print_tensor(tmp),end=' ')
    total=total+len(y)
    sm=sm+tf.reduce_sum(tf.cast(_, tf.int32))
  return sm/total

In [15]:
tf.keras.backend.clear_session()

inp=tf.keras.Input(shape=(21,1))
l=tf.keras.layers.LSTM(21,activation='relu',return_sequences=True)(inp)
out=tf.keras.layers.Dense(345,activation='softmax')(l)
#out=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,activation='softmax'))(l)

model1=tf.keras.Model(inp,out)
model1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 21, 21)            1932      
_________________________________________________________________
dense (Dense)                (None, 21, 345)           7590      
Total params: 9,522
Trainable params: 9,522
Non-trainable params: 0
_________________________________________________________________


In [16]:
model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
               metrics=[acc,'accuracy'])

model1.fit(trainx,trainy,batch_size=1024,epochs=10)

Epoch 1/10
135/135 [==============================] - 46s 341ms/step - loss: 3.6872 - accuracy: 0.3475
Epoch 2/10
135/135 [==============================] - 46s 338ms/step - loss: 2.2728 - accuracy: 0.4966
Epoch 3/10
135/135 [==============================] - 46s 339ms/step - loss: 1.8592 - accuracy: 0.5595
Epoch 4/10
135/135 [==============================] - 47s 347ms/step - loss: 1.6574 - accuracy: 0.5869
Epoch 5/10
135/135 [==============================] - 46s 340ms/step - loss: 1.5606 - accuracy: 0.5969
Epoch 6/10
135/135 [==============================] - 46s 338ms/step - loss: 1.4991 - accuracy: 0.6052
Epoch 7/10
135/135 [==============================] - 45s 335ms/step - loss: 1.4568 - accuracy: 0.6110
Epoch 8/10
135/135 [==============================] - 46s 339ms/step - loss: 1.4247 - accuracy: 0.6161
Epoch 9/10
135/135 [==============================] - 46s 340ms/step - loss: 1.3975 - accuracy: 0.6199
Epoch 10/10
135/135 [==============================] - 46s 338ms/step - l

In [17]:
# Prediction Accuracy
def pred_acc(model,out,inp):
  sm=0
  for i,p in zip(inp,out):
    #print(i.shape)
    i=i.reshape((1,21,1))
    _=np.argmax(model.predict(i),axis=2)
    _=_.reshape((21))
    p=p.reshape((21))
    _w=[fr_itow[i] for i in _]
    pw=[fr_itow[i] for i in p]
    print('\n\nActual:{}\nPredicted:{}'.format(_w,pw))
    tmp=_==p
    #print(np.sum(tmp),end=' ')
    sm=sm+np.sum(tmp)
  print('\n\nAccuracy is :{}'.format(sm/len(out)/21))
  return sm

c=pred_acc(model1,trainy[:10],trainx[:10])



Actual:['new', 'jersey', 'est', 'parfois', 'calme', 'en', "l'", 'et', 'il', 'est', 'est', 'en', 'en', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Predicted:['new', 'jersey', 'est', 'parfois', 'calme', 'pendant', "l'", 'automne', 'et', 'il', 'est', 'neigeux', 'en', 'avril', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Actual:['new', 'états', 'unis', 'est', 'parfois', 'chaud', 'en', 'en', 'et', 'il', 'est', 'parfois', 'parfois', 'en', 'en', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Predicted:['les', 'états', 'unis', 'est', 'généralement', 'froid', 'en', 'juillet', 'et', 'il', 'gèle', 'habituellement', 'en', 'novembre', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Actual:['la', 'est', 'est', 'parfois', 'en', "l'", 'et', 'il', 'il', 'est', 'parfois', 'en', 'en', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Predicted:['california', 'est', 'généralement', 'calme', 'en', 'mars', 'et', 

# RNN with Embedding 

In [23]:
tf.keras.backend.clear_session()

inp=tf.keras.Input(shape=(21))
l1=tf.keras.layers.Embedding(200,12)(inp)
l2=tf.keras.layers.LSTM(21,activation='relu',return_sequences=True)(l1)
out=tf.keras.layers.Dense(345,activation='sigmoid')(l2)
#out=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,activation='softmax'))(l)

model=tf.keras.Model(inp,out)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 21, 12)            2400      
_________________________________________________________________
lstm (LSTM)                  (None, 21, 21)            2856      
_________________________________________________________________
dense (Dense)                (None, 21, 345)           7590      
Total params: 12,846
Trainable params: 12,846
Non-trainable params: 0
_________________________________________________________________


In [26]:
trainx.shape

(137861, 21, 1)

In [24]:
train_x=trainx.reshape(137861,21)
train_y=trainy.reshape(137861,21)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
               metrics=[acc,'accuracy'])

model.fit(train_x,train_y,batch_size=1024,epochs=10)

Epoch 1/10
135/135 [==============================] - 38s 284ms/step - loss: 4.8437 - acc: 0.0677 - accuracy: 0.3458
Epoch 2/10
135/135 [==============================] - 38s 279ms/step - loss: 3.8700 - acc: 0.0483 - accuracy: 0.4230
Epoch 3/10
135/135 [==============================] - 38s 279ms/step - loss: 3.5736 - acc: 0.0582 - accuracy: 0.4248
Epoch 4/10
135/135 [==============================] - 38s 282ms/step - loss: 3.3845 - acc: 0.0540 - accuracy: 0.4220
Epoch 5/10
135/135 [==============================] - 38s 284ms/step - loss: 3.1420 - acc: 0.0785 - accuracy: 0.4368
Epoch 6/10
135/135 [==============================] - 40s 294ms/step - loss: 2.7692 - acc: 0.0932 - accuracy: 0.4451
Epoch 7/10
135/135 [==============================] - 38s 281ms/step - loss: 2.6092 - acc: 0.0983 - accuracy: 0.4489
Epoch 8/10
135/135 [==============================] - 38s 278ms/step - loss: 2.5352 - acc: 0.0865 - accuracy: 0.4439
Epoch 9/10
 21/135 [===>..........................] - ETA: 30s -

KeyboardInterrupt: ignored

In [20]:
# prediction 
c=pred_acc(model,trainy[:10],trainx[:10])



Actual:['la', 'les', 'est', 'est', 'est', 'est', 'est', 'est', 'est', 'est', 'est', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Predicted:['new', 'jersey', 'est', 'parfois', 'calme', 'pendant', "l'", 'automne', 'et', 'il', 'est', 'neigeux', 'en', 'avril', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Actual:['la', 'la', 'les', 'est', 'est', 'est', 'est', 'est', 'est', 'est', 'est', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Predicted:['les', 'états', 'unis', 'est', 'généralement', 'froid', 'en', 'juillet', 'et', 'il', 'gèle', 'habituellement', 'en', 'novembre', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Actual:['les', 'est', 'est', 'est', 'est', 'est', 'est', 'est', 'est', 'est', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
Predicted:['california', 'est', 'généralement', 'calme', 'en', 'mars', 'et',

# Bi-directional RNN 

In [ ]:
tf.keras.backend.clear_session()

inp=tf.keras.Input(shape=(21))
l1=tf.keras.layers.Embedding(200,12)(inp)
l2=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(21,activation='relu',return_sequences=True))(l1)
out=tf.keras.layers.Dense(345,activation='sigmoid')(l2)
#out=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,activation='softmax'))(l)

model2=tf.keras.Model(inp,out)
model2.summary()

# Bi-directional RNN with Embeddings

In [21]:
tf.keras.backend.clear_session()

inp=tf.keras.Input(shape=(21))
l1=tf.keras.layers.Embedding(200,12)(inp)
l2=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(21,activation='relu',return_sequences=True))(l1)
out=tf.keras.layers.Dense(345,activation='sigmoid')(l2)
#out=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,activation='softmax'))(l)

model2=tf.keras.Model(inp,out)
model2.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 21, 12)            2400      
_________________________________________________________________
bidirectional (Bidirectional (None, 21, 42)            5712      
_________________________________________________________________
dense (Dense)                (None, 21, 345)           14835     
Total params: 22,947
Trainable params: 22,947
Non-trainable params: 0
_________________________________________________________________


In [22]:
model2.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
               metrics=[acc,'accuracy'])

model2.fit(trainx,trainy,batch_size=1024,epochs=10)

Epoch 1/10
135/135 [==============================] - 44s 323ms/step - loss: 4.5972 - acc: 0.0708 - accuracy: 0.3974
Epoch 2/10
135/135 [==============================] - 44s 323ms/step - loss: 3.2737 - acc: 0.0849 - accuracy: 0.4353
Epoch 3/10
135/135 [==============================] - 44s 323ms/step - loss: 2.7311 - acc: 0.0935 - accuracy: 0.4523
Epoch 4/10
135/135 [==============================] - 44s 324ms/step - loss: 2.5466 - acc: 0.1022 - accuracy: 0.4568
Epoch 5/10
135/135 [==============================] - 44s 324ms/step - loss: 2.4298 - acc: 0.1094 - accuracy: 0.4558
Epoch 6/10
135/135 [==============================] - 45s 332ms/step - loss: 2.3935 - acc: 0.1084 - accuracy: 0.4522
Epoch 7/10
135/135 [==============================] - 44s 325ms/step - loss: 2.3580 - acc: 0.1113 - accuracy: 0.4557
Epoch 8/10
135/135 [==============================] - 44s 329ms/step - loss: 2.3078 - acc: 0.1107 - accuracy: 0.4562
Epoch 9/10
135/135 [==============================] - 44s 323ms/

In [ ]:
# RNN with embedding looks better among all above. 

# lets have guess with this model 

print('Sentence input:\n:',ftrain[0])
print('\nActual sentence output :\n',np.array([en_itow[i] for i in epad[0]]))
print('\n\nPredicted sentence output :\n',np.array([en_itow[i[0]] for i in np.argmax(model2.predict(fpad[0]),axis=2)]))

Sentence input:
: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

Actual sentence output :
 ['new' 'jersey' 'is' 'sometimes' 'quiet' 'during' 'autumn' 'and' 'it' 'is'
 'snowy' 'in' 'april' '<pad>' '<pad>' '<pad>' '<pad>' '<pad>' '<pad>'
 '<pad>' '<pad>']


Predicted sentence output :
 ['is' 'is' 'the' 'wanted' 'and' 'and' 'and' 'and' 'and' 'and' 'the'
 'during' 'dislikes' 'and' 'during' 'during' 'during' 'during' 'during'
 'during' 'during']


# Encoding and Decoding


In [ ]:
tf.keras.backend.clear_session()

inp=tf.keras.Input(shape=(21))
l1=tf.keras.layers.Embedding(346,12)(inp)
l2=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(21,activation='relu',return_sequences=True))(l1)
out=tf.keras.layers.Dense(201,activation='sigmoid')(l2)
#out=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,activation='softmax'))(l)

model2=tf.keras.Model(inp,out)
model2.summary()

In [ ]:
# Building encoder with bidirectional lstm
tf.keras.backend.clear_session()

class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder,self).__init__()
    #np=tf.keras.Input(shape=(64,21))
    self.em=tf.keras.layers.Embedding(346,12)
    self.enc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(21,activation='relu',return_sequences=True))
    self.d=tf.keras.layers.Dense(201,activation='sigmoid')
  
  def call(self,inputs):
    #x=inp(inputs)
    x=self.em(inputs)
    x=self.enc(x)
    x=self.d(x)
    return x

enc=Encoder()
#enc.build(input_shape=(64,21))
#enc.summary()


In [ ]:
enc.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
               metrics=[accuracy])

In [ ]:
enc.fit(train_input,train_output,batch_size=64,epochs=8)

Epoch 1/8


TypeError: ignored

In [ ]:
print('Sentence input:\n:',ftrain[5])
print('\nActual sentence output :\n',np.array([en_itow[i] for i in epad[5]]))
print('\n\nPredicted sentence output :\n',np.array([en_itow[i[0]] for i in np.argmax(enc.predict(fpad[5]),axis=2)]))

Sentence input:
: son fruit préféré est l'orange , mais mon préféré est le raisin .

Actual sentence output :
 ['his' 'favorite' 'fruit' 'is' 'the' 'orange' 'but' 'my' 'favorite' 'is'
 'the' 'grape' '<pad>' '<pad>' '<pad>' '<pad>' '<pad>' '<pad>' '<pad>'
 '<pad>' '<pad>']


Predicted sentence output :
 ['his' 'least' 'least' 'next' 'visit' 'least' 'my' 'least' 'next' 'least'
 'visit' 'least' 'least' 'least' 'least' 'least' 'least' 'least' 'least'
 'least' 'least']


In [ ]:
#calculating actual accuracy

train_output.shape

(130000, 21)

In [ ]:
def acc(model,out,inp):
  cnt=0
  cn=0
  for i,p in (inp,out):
    _=np.argmax(model.predict(i),axis=2)
    for j in range(len(i)):
      if p[j]==_[j]:
        cnt=cnt+1
    cn=cn+1
    if cn%1000==0:
      print(cn)
  print('\n\nAccuracy is :{}'.format(cnt/100/21))
  return cnt

c=acc(enc,train_output[:100],train_input[:100])

ValueError: ignored

In [ ]:
import keras.backend as K
import numpy as np


In [ ]:
def acc(y_true,y_pred):
  sm=0
  for i,j in zip(y_true,y_pred):
    _=i==j
    sm=sm+np.sum(_)
  return sm/len(y_true)/21


In [ ]:
a=tf.constant([1,2,3,4,5])
b=tf.constant([8,7,3,4,5])

In [ ]:
np.array([1])

NameError: ignored

In [ ]:
a

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 2, 3, 4, 5], dtype=int32)>